In [1]:
import io
import json
import os
from pprint import pp

import pandas as pd
from termcolor import colored

In [2]:
def get_directories(path):
    dirs = [dir for dir in os.listdir(path) if os.path.isdir(os.path.join(path, dir))]
    return dirs

In [3]:
def load_jsonl(path):
    lines = []
    with open(path, "r") as file:
        for line in file:
            lines.append(json.loads(line))
    return lines

In [4]:
def get_model_dir(model_dirs, gen_mode, model_name_escaped, dedup_str, n_tool_calls):
    for model_dir in model_dirs:
        if gen_mode == "structured" and "unstructured" in model_dir:
            continue

        if gen_mode in model_dir and model_name_escaped in model_dir and dedup_str in model_dir and n_tool_calls in model_dir:
            return model_dir

In [5]:
def index_it(idx, questions, solutions, generations, scores, idx_type="error"):

    if idx_type == "absolute":
        question = questions[idx]
        solution = solutions[idx]
        generation = generations[idx]
        score = [score for i, score in enumerate(scores) if i > 0 and score["id"] == idx]
    elif idx_type == "error":
        score = scores[idx+1]
        abs_idx = score["id"] - 1
        question = questions[abs_idx]
        solution = solutions[abs_idx]
        generation = generations[abs_idx]
    else:
        raise ValueError("Wrong idx_type.")

    return question, solution, generation, score

In [6]:
def extract_values( question, solution, generation, score):
    user_query = question["question"]
    tools = question["function"]
    if not isinstance(tools, list):
        tools = [tools]

    raw_result = generation["result"]
    messages = generation["messages"]
    tool_calls = generation["tool_calls"]
    n_tool_calls = generation["n_tool_calls"]
    valid = score["valid"]
    error = score["error"]
    error_type = score["error_type"]
    decoded_result = score["model_result_decoded"]

    result = {
        "user_query": user_query,
        "tools": tools,
        "valid": valid,
        "tool_calls": tool_calls,
        "solution": solution,
        "error": error,
        "error_type": error_type,
        "raw_result": raw_result,
        "decoded_result": decoded_result,
        "messages": messages,
        "n_tool_calls": n_tool_calls
    }

    return result

In [7]:
# Inputs
bfcl_category = "simple"

dedup = True
gen_mode = "structured"
n_tool_calls = "auto"
model_name = "databricks/dbrx-instruct"
out_dir = "./outputs"

spec = {
    "bfcl_category": bfcl_category,
    "gen_mode": gen_mode,
    "n_tool_calls": n_tool_calls,
    "model_name": model_name,
    "dedup": dedup,
}

In [8]:
# Get paths
test_file = f"gorilla_openfunctions_v1_test_{bfcl_category}.json"
questions_path = os.path.join("./data", test_file)
solutions_path = os.path.join("./data/possible_answer", test_file)


model_name_escaped = model_name.replace("/", "_")
dedup_str = "dedup" if dedup else ""
model_dirs = get_directories(out_dir)
model_dir = get_model_dir(model_dirs, gen_mode, model_name_escaped, dedup_str, n_tool_calls)
generations_path = os.path.join(out_dir, model_dir, "generations", test_file)
scores_path = os.path.join(out_dir, model_dir, "scores", bfcl_category + "_score.json")

In [9]:
# Load all files
questions = load_jsonl(questions_path)
solutions = load_jsonl(solutions_path)
generations = load_jsonl(generations_path)
scores = load_jsonl(scores_path)

In [10]:
idx = 0
idx_type = "error"
items = index_it(idx, questions, solutions, generations, scores, idx_type=idx_type)
question, solution, generation, score = tuple(items)
values = extract_values(question, solution, generation, score)

result = spec | values | {"idx": idx, "idx_type": idx_type}

In [11]:
def pretty_print_conversation(messages):
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "function": "magenta",
    }

    for message in messages:
        if message["role"] == "system":
            print(colored(f"system: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "user":
            print(colored(f"user: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and message.get("function_call"):
            print(colored(f"assistant: {message['function_call']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and not message.get("function_call"):
            print(colored(f"assistant: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "function":
            print(colored(f"function ({message['name']}): {message['content']}\n", role_to_color[message["role"]]))


In [12]:
result.keys()

dict_keys(['bfcl_category', 'gen_mode', 'n_tool_calls', 'model_name', 'dedup', 'user_query', 'tools', 'valid', 'tool_calls', 'solution', 'error', 'error_type', 'raw_result', 'decoded_result', 'messages', 'idx', 'idx_type'])

In [22]:
def pp_result(result, width=200, verbose=0):
    print("-"*120)
    dedup_str = "-dedup" if dedup else ""
    print(f"Example {result['idx']} of {result['bfcl_category']} BFCL\t\t\t\t\t({result['model_name']}-{result['gen_mode']} with {result['n_tool_calls']}{dedup_str})")
    print("-"*120, end="\n\n")

    color_map = {}
    print(colored(f"User Query:\n{result['user_query']}", "yellow"), end="\n\n")

    print(colored(f"Valid: {result['valid']}", "blue"), end="\n\n")

    print(colored(f"Error:\n{result['error']}\n{result['error_type']}", "blue"), end="\n\n")

    print(colored("My Tool Calls:", "red"))
    for tool_call in result['tool_calls']:
        print(colored(tool_call, "red"))
    print()

    print(colored("Correct Tool Calls:", "green"))
    for tool_name, tool_args in result['solution'].items():
        print(colored(f"{{'tool_name': '{tool_name}', 'tool_arguments': {tool_args}}}", "green"))
    print()

    print(colored(f"Raw Result:\n{result['raw_result']}", "yellow"), end="\n\n")

    if verbose >= 1:

        print(colored(f"Decoded Result:\n{result['decoded_result']}", "blue"), end="\n\n")

        print("Given Tool Calls:")
        for tool in result["tools"]:
            pp(tool, width=width)
        print()

    if verbose >= 2:
        print("Messages:")
        pretty_print_conversation(result['messages'])


pp_result(result)

------------------------------------------------------------------------------------------------------------------------
Example 0 of simple BFCL					(databricks/dbrx-instruct-structured with auto-dedup)
------------------------------------------------------------------------------------------------------------------------

User Query:
Find the area of a triangle with a base of 10 units and height of 5 units.

Valid: False

Error:
['Wrong number of functions.']
simple_function_checker:wrong_count

My Tool Calls:
{'tool_name': 'calculate_triangle_area', 'tool_arguments': {'base': 10, 'height': 5}}
{'tool_name': 'calculate_triangle_area', 'tool_arguments': {'base': 10, 'height': 5, 'unit': 'units'}}

Correct Tool Calls:
{'tool_name': 'calculate_triangle_area', 'tool_arguments': {'base': [10], 'height': [5], 'unit': ['units', '']}}

Raw Result:
[calculate_triangle_area(base=10, height=5), calculate_triangle_area(base=10, height=5, unit='units')]

